In [1]:
from utils.train_utils import *
from logic_data.utils import *

In [2]:
if __name__ == '__main__':
    is_notebook = False
    try:
        cmd = argparse.ArgumentParser('The testing components of')
        cmd.add_argument('--gpu', default=-1, type=int, help='use id of gpu, -1 if cpu.')
        cmd.add_argument('--train_batch_size', default=128, type=int, help='training batch size')
        cmd.add_argument('--eval_batch_size', default=128, type=int, help='training batch size')
        cmd.add_argument('--lr', default=0.01, type=float, help='learning rate')
        cmd.add_argument('--data_path', required=True, type=str, help='path to the training corpus')
        cmd.add_argument('--train_data_path', required=True, type=str, help='path to the training corpus')
        cmd.add_argument('--test_data_path', required=True, type=str, help='path to the training corpus')
        cmd.add_argument(
            '--encoder_config_path', 
            type=str, help='path to the encoder config'
        )
        cmd.add_argument(
            '--decoder_config_path', 
            type=str, help='path to the decoder config'
        )
        cmd.add_argument('--max_seq_len', default=512, type=int)
        cmd.add_argument('--seed', default=42, type=int)
        cmd.add_argument('--gradient_accumulation_steps', default=1, type=int)
        cmd.add_argument('--output_dir', required=True, type=str, help='save dir')
        cmd.add_argument('--local_rank', default=-1, type=int, help='multi gpu training')
        cmd.add_argument('--epochs', default=10, type=int, help='training epochs')
        cmd.add_argument('--model_path', type=str, required=False, default=None)
        cmd.add_argument('--warm_up', type=float, default=0.1)
        cmd.add_argument('--is_wandb', default=False, action='store_true')
        cmd.add_argument('--log_step', default=10, type=int)
        cmd.add_argument('--valid_steps', default=500, type=int)
        cmd.add_argument('--early_stopping', default=5, type=int)
        cmd.add_argument('--device', default="cuda", type=str, help='')
        cmd.add_argument('--do_prealign_eval', default=False, action='store_true')
        cmd.add_argument('--do_align', default=False, action='store_true')
        cmd.add_argument('--do_eval', default=False, action='store_true')
        cmd.add_argument('--do_test', default=False, action='store_true')
        
        cmd.add_argument('--n_training_program', default=5, type=int)
        cmd.add_argument('--n_fewshot', default=6, type=int)
        cmd.add_argument('--aligning_layer_n', default=0, type=int)
        
        args = cmd.parse_args(sys.argv[1:])
    except:
        is_notebook = True
        parser = argparse.ArgumentParser()
        args = parser.parse_args([])
        args.gpu = 1
        args.train_batch_size = 64
        args.eval_batch_size = 64
        args.gradient_accumulation_steps = 2
        args.lr = 1e-4
        args.data_path = "./logic_data"
        args.train_data_path = ""
        args.test_data_path = ""
        args.encoder_config_path = None
        args.decoder_config_path = None
        args.max_seq_len = 512
        args.seed = 66
        args.output_dir = "./results_notebook/"
        args.epochs = 10
        args.warm_up = 0.1
        args.is_wandb = False
        args.log_step = 10
        args.valid_steps = 100 # -1 not do training eval!
        args.early_stopping = 999 # large == never early stop!
        args.device = "cuda:0"
        args.do_prealign_eval = True # do it once at least!
        args.do_align = True
        args.do_eval = True
        args.do_test = True
        args.model_path = "./results_notebook/logic_pipeline.model.gpt2.n_rule.9.n_shot.10.seed.66/model-last/"
        # args.model_path = None
        
        # alignment search setting
        args.aligning_layer_n = 0
        args.aligning_basis_n_per_variable = 600
        args.aligning_var_n = 1
        
        print("Using in a notebook env.")

Using in a notebook env.


usage: The testing components of [-h] [--gpu GPU]
                                 [--train_batch_size TRAIN_BATCH_SIZE]
                                 [--eval_batch_size EVAL_BATCH_SIZE] [--lr LR]
                                 --data_path DATA_PATH --train_data_path
                                 TRAIN_DATA_PATH --test_data_path
                                 TEST_DATA_PATH
                                 [--encoder_config_path ENCODER_CONFIG_PATH]
                                 [--decoder_config_path DECODER_CONFIG_PATH]
                                 [--max_seq_len MAX_SEQ_LEN] [--seed SEED]
                                 [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                                 --output_dir OUTPUT_DIR
                                 [--local_rank LOCAL_RANK] [--epochs EPOCHS]
                                 [--model_path MODEL_PATH] [--warm_up WARM_UP]
                                 [--is_wandb] [--log_step LOG_STEP]
          

In [3]:
results = []

In [4]:
training_clauses = [
    '( c != a ) or ( a == b )',
#     '( c == a ) or ( a != b )',
#     '( c != a ) and ( a == b )',
#     '( c == a ) or ( b != c )',
#     '( c == a ) or ( b == c )',
#     '( b != a ) and ( b != c )',
#     '( c != a ) or ( a != b )',
#     '( c == a ) and ( b != c )',
#     '( c != a ) and ( b != c )',
#     '( c == a ) or ( a == b )',
#     '( c != a ) and ( a != b )'
]
align_type = "left_aligment"
if align_type == "function_aligment":
    source_clauses_pool = [
        '( c != a ) or ( a != b )',
        '( c == a ) and ( b != c )',
        '( c != a ) and ( b != c )',
        '( c == a ) or ( a == b )',
         '( c != a ) and ( a != b )',
        '( c != a ) or ( a == b )',
        '( c == a ) or ( a != b )',
        '( c != a ) and ( a == b )',
        '( c == a ) or ( b != c )',
        '( c == a ) or ( b == c )',
        '( b != a ) and ( b != c )',
    ]
else:
    source_clauses_pool = [
        '( c != a ) or ( a == b )'
    ]
level = "l2"
shared_train = True if level == "l1" else False
n_fewshot = 10
n_examples = n_fewshot + 1
n_training_examples = 5000
n_testing_examples = 1000
saving_models = True

for seed in [66,]: # [42, 66, 77, 88, 99]
    args.seed = seed
    for clauses in training_clauses:
        set_seed(args.seed)

        clauses_no_space = clauses.replace(" ", "")
        print("aligning for clauses: ", clauses)
        print("seed: ", seed)
        
        for source_clauses in source_clauses_pool:
        # we generate the dataset onfly
            if align_type == "left_aligment":
                sampler_func = left_aligment_sampler
                source_clauses = None
            elif align_type == "whole_aligment":
                sampler_func = whole_aligment_sampler
                print("aligning with source clauses: ", source_clauses)
            elif align_type == "function_aligment":
                sampler_func = function_aligment_sampler
                print("aligning with source clauses: ", source_clauses)

            aligment_data = sampler_func(
                clauses, n_training_examples, n_examples = n_examples, shared_train=shared_train, source_clauses=source_clauses
            )
            train_data = {
                "base_input_ids" : aligment_data[0],
                "base_output_ids" : aligment_data[1],
                "source_input_ids" : aligment_data[3],
                "source_output_ids" : aligment_data[4],
                "counterfacut_output_ids": aligment_data[6],
                "clauses" : aligment_data[2],
                "intervention_ids": [0 for i in range(len(aligment_data[0]))]
            }
            aligment_sampled_data = sampler_func(
                clauses, n_testing_examples, n_examples = n_examples, shared_train=shared_train, source_clauses=source_clauses
            )
            test_data = {
                "base_input_ids" : aligment_sampled_data[0],
                "base_output_ids" : aligment_sampled_data[1],
                "source_input_ids" : aligment_sampled_data[3],
                "source_output_ids" : aligment_sampled_data[4],
                "counterfacut_output_ids": aligment_sampled_data[6],
                "clauses" : aligment_sampled_data[2],
                "intervention_ids": [0 for i in range(len(aligment_sampled_data[0]))]
            }

            model_name = args.model_path.strip("/").split("/")[-2]
            run_name = f"{model_name}.clauses.{clauses_no_space}.align.{align_type}.seed.{args.seed}"
            logger = logging.getLogger()

            train_dataset = Dataset.from_dict(
                {
                    "input_ids": train_data["base_input_ids"], 
                    "labels": train_data["base_output_ids"],
                    "source_input_ids": train_data["source_input_ids"], 
                    "counterfactual_labels": train_data["counterfacut_output_ids"],
                    "intervention_ids": train_data["intervention_ids"],
                }
            ).with_format("torch")
            train_dataloader = DataLoader(train_dataset, batch_size=args.train_batch_size)

            test_dataset = Dataset.from_dict(
                {
                    "input_ids": test_data["base_input_ids"], 
                    "labels": test_data["base_output_ids"],
                    "source_input_ids": test_data["source_input_ids"], 
                    "counterfactual_labels": test_data["counterfacut_output_ids"],
                    "intervention_ids": test_data["intervention_ids"],
                }
            ).with_format("torch")
            test_dataloader = DataLoader(test_dataset, batch_size=args.eval_batch_size)

            # Model
            torch.cuda.empty_cache()

            configuration = GPT2Config.from_pretrained(os.path.join(args.data_path, "decoder_config.json"))

            if "logic" in model_name:
                arity = 3
            
            # the following lines are for the neuron population.
            start_idx = 1 + (arity + 4) * int(n_fewshot)
            end_idx = start_idx + (arity+1)
            alignment_config = {
                "layer" : args.aligning_layer_n,
                "token_range" : [start_idx, end_idx] # this is kind of fixed?
            }
            
            if args.aligning_var_n == 1:
                intervention_config = {
                    0: [[0, args.aligning_basis_n_per_variable]]
                }
            elif args.aligning_var_n == 2:
                intervention_config = {
                    0: [[0, args.aligning_basis_n_per_variable]],
                    1: [[args.aligning_basis_n_per_variable, 2*args.aligning_basis_n_per_variable]],
                    2: [[0, args.aligning_basis_n_per_variable], [args.aligning_basis_n_per_variable, 2*args.aligning_basis_n_per_variable]]
                }
            logging.info(f"intervention_config = {intervention_config}")
            logging.info(f"alignment_config = {alignment_config}")

            model = AlignableGPT2LMHeadModel(configuration, alignment_config=alignment_config)
            if args.model_path is not None:
                logging.info("Loading pretrained model.")
                raw_weights = torch.load(os.path.join(args.model_path, 'pytorch_model.bin'))
                model.load_state_dict(raw_weights, strict=False)

            # we need to set off gradients!
            for name, param in model.named_parameters():
                if "rotate_layer" not in name:
                    param.requires_grad = False

            device = torch.device(args.device)
            if "cuda:" not in args.device:
                n_gpu = torch.cuda.device_count()
                logging.info(f'__Number CUDA Devices: {n_gpu}')
            else:
                n_gpu = 1
                logging.info(f'__Number CUDA Devices: {n_gpu}')

            if n_gpu > 1:
                model = torch.nn.DataParallel(model)
            _ = model.to(device)

            t_total = int(len(train_dataloader) * args.epochs)

            warm_up_steps = args.warm_up * t_total
            optimizer = torch.optim.AdamW(
                model.parameters(), lr=args.lr
            )
            scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warm_up_steps,
                                                        num_training_steps=t_total)
            is_master = True                                    
            if not os.path.exists(args.output_dir) and is_master:
                os.mkdir(args.output_dir)
            os.environ["WANDB_PROJECT"] = f"ToM-DAS"

            output_dir = os.path.join(args.output_dir, run_name)
            if args.do_align and args.is_wandb:
                import wandb
                run = wandb.init(
                    project="ToM-DAS-GPT2", 
                    entity="wuzhengx",
                    name=run_name,
                )
                wandb.config.update(args)
            if not os.path.exists(args.output_dir) and is_master:
                os.mkdir(args.output_dir)

            prealign_acc = None
            test_acc = None

            if args.do_prealign_eval:
                # before doing alignment, we need to check factual performance on the dataset.
                total_count = 0
                correct_count = 0
                if args.do_eval:
                    _ = model.eval()
                    epoch_iterator = tqdm(test_dataloader, desc="Iteration", position=0, leave=True)
                    for step, inputs in enumerate(epoch_iterator):
                        input_ids = inputs['input_ids'].to(device)
                        labels = inputs['labels'].to(device)
                        outputs = model(input_ids=input_ids)
                        actual_test_labels = labels[:, -3]
                        pred_test_labels = torch.argmax(outputs.logits[:, -4], dim=-1)
                        correct_labels = (actual_test_labels==pred_test_labels)

                        total_count += len(correct_labels)
                        correct_count += correct_labels.sum().tolist()

                        current_acc = round(correct_count/total_count, 2)
                        epoch_iterator.set_postfix({'acc': current_acc})
                prealign_acc = round(correct_count/total_count, 2)

            aligner = LogicSolverAligner(
                model, device=device, 
                logger=logger,
                is_master=saving_models, # making it False will prevent it from saving models! 
                n_gpu=n_gpu,
                is_wandb=args.is_wandb, 
                model_name=model_name,
                intervention_config=intervention_config
            )
            num_params = count_parameters(model)
            logging.info(f'Number of {model_name} model params: {num_params}') 

            # Train
            if args.do_align:
                logging.info(f"OUTPUT DIR: {output_dir}")
                aligner.train(
                    train_dataloader, test_dataloader,
                    optimizer, scheduler, 
                    log_step=args.log_step, valid_steps=args.valid_steps,
                    output_dir=output_dir, epochs=args.epochs, 
                    gradient_accumulation_steps=args.gradient_accumulation_steps,
                )

            if args.is_wandb:
                wandb.finish()

            # Test
            if args.do_test: 
                # Model
                torch.cuda.empty_cache()
                total_count = 0
                correct_count = 0
                aligner.model.eval()
                epoch_iterator = tqdm(test_dataloader, desc="Iteration", position=0, leave=True)
                for step, inputs in enumerate(epoch_iterator):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(device)
                    if aligner.preload_intervention_corr is not None:
                        intervention_corr = aligner.preload_intervention_corr.expand(
                            inputs['input_ids'].shape[0],-1
                        ).to(device)
                    else:
                        assert False # not implemented

                    # aligning forward!
                    source_hidden_states = aligner.model(
                       input_ids=inputs['source_input_ids']
                    ).rotated_hidden_states
                    outputs = aligner.model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_corr=intervention_corr,
                        labels=inputs['counterfactual_labels']
                    )

                    actual_test_labels = inputs['counterfactual_labels'][:, -3]
                    pred_test_labels = torch.argmax(outputs.logits[:, -4], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

                    current_acc = round(correct_count/total_count, 2)
                    epoch_iterator.set_postfix({'acc': current_acc})
                test_acc = round(correct_count/total_count, 2)
            results += [[clauses, source_clauses, seed, prealign_acc, test_acc]]

aligning for clauses:  ( c != a ) or ( a == b )
seed:  66


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4152.85it/s]
INFO:root:intervention_config = {0: [[0, 600]]}
INFO:root:alignment_config = {'layer': 0, 'token_range': [71, 75]}
INFO:root:Loading pretrained model.
INFO:root:__Number CUDA Devices: 1
Iteration: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 12.14it/s, acc=0.99]
INFO:root:Number of logic_pipeline.model.gpt2.n_rule.9.n_shot.10.seed.66 model params: 1440000
INFO:root:OUTPUT DIR: ./results_notebook/logic_pipeline.model.gpt2.n_rule.9.n_shot.10.seed.66.clauses.(c!=a)or(a==b).align.left_aligment.seed.66
Epoch: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [5]:
result_df = pd.DataFrame(results, columns = [
    'clauses', 'source_clauses', 'seed', 'prealign_acc', 'test_acc'
])

In [7]:
result_df.groupby(
    ['clauses', 'source_clauses'], as_index=False
).mean()[['clauses', 'source_clauses', 'prealign_acc', 'test_acc']].style.set_properties(**{'text-align': 'left'})

,clauses,source_clauses,prealign_acc,test_acc
0,( c != a ) or ( a == b ),( b != a ) and ( b != c ),0.990000,0.760000
1,( c != a ) or ( a == b ),( c != a ) and ( a != b ),0.990000,0.820000
2,( c != a ) or ( a == b ),( c != a ) and ( a == b ),0.990000,0.730000
3,( c != a ) or ( a == b ),( c != a ) and ( b != c ),0.990000,0.810000
4,( c != a ) or ( a == b ),( c != a ) or ( a != b ),0.990000,0.840000
5,( c != a ) or ( a == b ),( c != a ) or ( a == b ),0.990000,0.990000
6,( c != a ) or ( a == b ),( c == a ) and ( b != c ),0.990000,0.730000
7,( c != a ) or ( a == b ),( c == a ) or ( a != b ),0.990000,0.860000
8,( c != a ) or ( a == b ),( c == a ) or ( a == b ),0.990000,0.770000
9,( c != a ) or ( a == b ),( c == a ) or ( b != c ),0.990000,0.930000
